# Runtime (Polyglot)

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.2.4/lib/netstandard2.0/Argu.dll"

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common

In [ ]:
//// test

open SpiralFileSystem.Operators

## parseArgs

In [ ]:
let inline parseArgs<'T when 'T :> Argu.IArgParserTemplate> args =
    let assemblyName = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    let errorHandler : Argu.IExiter =
        if [ "Microsoft.DotNet.Interactive.App"; "dotnet-repl" ] |> List.contains assemblyName
        then Argu.ExceptionExiter ()
        else Argu.ProcessExiter (function Argu.ErrorCode.HelpText -> None | _ -> Some System.ConsoleColor.Red)

    let parser =
        Argu.ArgumentParser.Create<'T> (
            programName = $"{assemblyName}{SpiralPlatform.get_executable_suffix ()}",
            errorHandler = errorHandler
        )

    parser.ParseCommandLine args

In [ ]:
//// test

[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.ExactlyOnce; Argu.ArguAttributes.Last>]
        Paths of paths : string list

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Paths _ -> nameof Paths

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

"USAGE: dotnet-repl [--help] <paths>...

PATHS:

    <paths>...            Paths

OPTIONS:

    --help                display this list of options.
"


In [ ]:
//// test

fun () -> parseArgs<Arguments> [||] |> ignore
|> _throwsC (fun ex _ ->
    SpiralSm.format_exception ex
    |> _stringContains "Argu.ArguParseException: ERROR: missing parameter '<paths>...'."
)

<fun:it@3-3>

"Argu.ArguParseException: ERROR: missing parameter '<paths>...'.
USAGE: dotnet-repl [--help] <paths>...

PATHS:

    <paths>...            Paths

OPTIONS:

    --help                display this list of options.
"



In [ ]:
let inline parseAllArgs<'T when 'T :> Argu.IArgParserTemplate> args =
    args
    |> parseArgs<'T>
    |> fun results -> results.GetAllResults ()

In [ ]:
//// test

[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.ExactlyOnce; Argu.ArguAttributes.Last>]
        Paths of paths : string list

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Paths _ -> nameof Paths

parseAllArgs<Arguments> [| "a b"; "c" |]
|> _assertEqual [ Arguments.Paths [ "a b"; "c" ] ]

[Paths ["a b"; "c"]]



In [ ]:
let inline parseArgsMap<'T when 'T :> Argu.IArgParserTemplate> args =
    args
    |> parseAllArgs<'T>
    |> List.groupBy CommonFSharp.getUnionCaseName<'T>
    |> Map.ofList

In [ ]:
//// test

parseArgsMap<Arguments> [| "a b"; "c" |]
|> _assertEqual (
    [ nameof Arguments.Paths, [ Arguments.Paths [ "a b"; "c" ] ] ]
    |> Map.ofList
)

map [("Paths", [Paths ["a b"; "c"]])]

